In [7]:
import sys
from pathlib import Path

# add parent folder to the path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

___

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader # load batches to the network

from feito.trainer import BasecallerTrainer as Trainer
from feito.models import SimpleNet, Rodan
from feito.loss_functions import ctc_label_smoothing_loss
from feito.dataloader import DatasetONT
from feito.callbacks import CSVLogger

___
PARAMS

In [9]:
EPOCHS=5
BATCH_SIZE=16

### Training network
Model, loss function and optimizer

In [10]:
# network to use
model=SimpleNet()
# model=Rodan()
model_output_len = model.output_len # another way to obtain the output of the model https://github.com/biodlab/RODAN/blob/029f7d5eb31b11b53537f13164bfedee0c0786e4/model.py#L317
loss_fn = nn.CTCLoss() #ctc_label_smoothing 
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)


Load dataset

In [11]:
# dataset
dataset_train = DatasetONT(recfile="../data/subsample_train.hdf5", output_network_len=model_output_len) #simplenet_output_len)
dataset_val   = DatasetONT(recfile="../data/subsample_val.hdf5", output_network_len=model_output_len) #simplenet_output_len)

dataloader_train = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=False)

Callbacks

In [12]:
csv_logger=CSVLogger(list_vars=["train_loss","val_loss"], out_file="output/training/metrics.csv", overwrite=False)

Train

In [13]:
trainer=Trainer(
    model=SimpleNet(),
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    train_loader=dataloader_train,
    validation_loader=dataloader_val,
    criterion=loss_fn,
    optimizer=optimizer,
    callbacks=[csv_logger]
)

In [14]:
trainer.fit(epochs=EPOCHS)

Training Epoch: 1 | Batch: 0/13:   0%|                                                     | [00:00][W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.
Training Epoch: 1 | Batch: 12/13: 100%|███████████████████████████████████████| [00:04, loss=2.4205]
Validate Epoch: 1 | Batch: 12/13: 100%|███████████████████████████████████████| [00:01, loss=2.5431]


calling callback


Training Epoch: 2 | Batch: 12/13: 100%|███████████████████████████████████████| [00:04, loss=2.5232]
Validate Epoch: 2 | Batch: 12/13: 100%|███████████████████████████████████████| [00:02, loss=2.5431]


calling callback


Training Epoch: 3 | Batch: 12/13: 100%|███████████████████████████████████████| [00:04, loss=2.4848]
Validate Epoch: 3 | Batch: 12/13: 100%|███████████████████████████████████████| [00:01, loss=2.5431]


calling callback


Training Epoch: 4 | Batch: 12/13: 100%|███████████████████████████████████████| [00:04, loss=2.5486]
Validate Epoch: 4 | Batch: 12/13: 100%|███████████████████████████████████████| [00:01, loss=2.5431]


calling callback


Training Epoch: 5 | Batch: 12/13: 100%|███████████████████████████████████████| [00:04, loss=2.5499]
Validate Epoch: 5 | Batch: 12/13: 100%|███████████████████████████████████████| [00:01, loss=2.5431]


calling callback
Done!
